In [24]:
from pynq import Overlay
import asyncio
import numpy as np
import pynq
import time

overlay = Overlay("/home/xilinx/IPBitFile/Unmixing.bit")
# overlay.download()

AE = overlay.Abundance_Estimator_0
MVHT = overlay.MVHT_0

# pynq.ps.Clocks.fclk0_mhz = 300

Fp_hsi = open("/home/xilinx/jupyter_notebooks/hsi.txt", 'r')

hsi_ori = np.zeros((65536, 8))
hsi_ds = np.zeros((4096, 8))

for i in range(65536):
    for j in range(8):
        line_hsi = Fp_hsi.readline()
        hsi_ori[i, j] = float(line_hsi[:-1])
        
for i in range(4096):
    hsi_ds[i,:] = hsi_ori[i*16, :]

for i in range(8):
    for j in range(4096):
        MVHT.write(0x4000 + 4*(i*4096+j), int(hsi_ds[j, i]*2**22))
    
# set interrupt for done signal
MVHT.write(0x0008, 1)
# enable globel interrupt
MVHT.write(0x0004, 1)
# start
MVHT.write(0x0000, 1)

# t0 = time.time()

while True:
    if (MVHT.read(0x000c) == 1):
        vtx_idx = np.zeros(8, int)
        for idx in range(8):
            vtx_idx[idx] = int(MVHT.read(0x24000 + 4*idx))
            print(str(vtx_idx[idx]))
        break

        

# async def MVHT_intr_handler(MVHT):
#     while True:
#         await MVHT.interrupt.wait()
#         print('intr received from ' + str(MVHT.read(0x000c)))
#         if (MVHT.read(0x000c) == 1):
#             vtx_idx = np.zeros(8)
#             for idx in range(8):
#                 vtx_idx[idx] = MVHT.read(0x24000 + 4*idx)
#                 print(str(vtx_idx[idx]))
#         if (MVHT.read(0x000c) & 0x1):
#             MVHT.write(0x000c, 1)
            
            
async def AE_intr_handler(AE):
    while True:
        await AE.interrupt.wait()
        print('intr received from ' + str(AE.read(0x000c)))
        if (AE.read(0x000c) == 1):
            abundance = np.zeros((8,64))
            for i in range(8):
                for j in range(64):
                    abundance[i, j] = AE.read(0x1800 + 4*(i*64+j)) / 2**22
                    print(str(abundance[i, j]))
            break

# # get EventLoop:
# loop = asyncio.get_event_loop()
# # run coroutine
# loop.run_until_complete(MVHT_intr_handler(MVHT))
# loop.close()

# vtx_idx = [242, 688, 183, 66, 3145, 2277, 532, 377]


endmember = np.zeros((7,8))
hsi_batch = np.zeros((7,64))
for i in range(8):
    endmember[:, i] = hsi_ori[vtx_idx[i]*16,:-1].T

for batch in range(1024):
    hsi_batch = hsi_ori[batch*64:(batch+1)*64,:-1].T
    for j in range(7):
        for k in range(8):
            AE.write(0x1000 + 4*(j*8+k), int(endmember[j, k]*2**22))
        for k in range(64):
            AE.write(0x0800 + 4*(j*64+k), int(hsi_batch[j, k]*2**22))
            
    # set interrupt for done signal
    AE.write(0x0008, 1)
    # enable globel interrupt
    AE.write(0x0004, 1)
    # start
    AE.write(0x0000, 1)
    
    # get EventLoop:
    loop = asyncio.get_event_loop()
    # run coroutine
    loop.run_until_complete(AE_intr_handler(AE))
#     loop.close()
    
    # set interrupt for done signal
    AE.write(0x0008, 0)
    # enable globel interrupt
    AE.write(0x0004, 0)
    # start
    AE.write(0x0000, 0)
    

# print('time:', time.time() - t0)


242
688
183
1168
2387
82
2277
532
intr received from 1
0.114082813263
0.122836351395
0.131536960602
0.122903585434
0.174998283386
0.169840812683
0.130173683167
0.136184453964
0.158291578293
0.137386798859
0.145278930664
0.196561813354
0.227564811707
0.2259349823
0.208570241928
0.186896800995
0.201103210449
0.233337879181
0.202917575836
0.18846821785
0.15750837326
0.127938508987
0.164669036865
0.706647157669
0.359102010727
0.280426502228
0.0118975639343
0.17607665062
0.344566345215
0.863315582275
0.418147087097
0.323560476303
0.323078870773
0.313997507095
0.300575256348
0.306904315948
0.231827020645
0.131268501282
0.0988719463348
0.101434707642
0.0963535308838
0.0865590572357
0.0918226242065
0.357372522354
0.466752767563
0.475341320038
0.530028820038
0.550505876541
0.648631811142
0.272953987122
0.0713567733765
0.0612156391144
0.331595182419
0.568508386612
0.139434337616
0.179606437683
0.121968984604
0.112155675888
0.0661604404449
0.0722262859344
0.230662822723
0.162310838699
0.133952856

0.24849319458
0.0240077972412
0.0423092842102
0.322585105896
0.294743061066
0.0941290855408
0.005202293396
0.0185713768005
0.0103569030762
0.00355839729309
0.00815796852112
0.00938272476196
0.0963835716248
0.106422662735
0.105278253555
0.131987571716
0.163029909134
0.0418481826782
0.00133156776428
0.0600860118866
0.122673749924
0.228173732758
0.219557762146
0.095543384552
0.0523362159729
0.0149953365326
0.00365519523621
0.0660350322723
0.126477718353
0.168376684189
0.112032413483
0.0641160011292
0.0083167552948
0.00647759437561
0.00659346580505
0.00345134735107
0.00317525863647
0.0
0.0
0.0
9.53674316406e-05
0.0177838802338
0.0557298660278
0.120325565338
0.112070322037
0.104164361954
0.101376771927
0.00372290611267
0.0
0.0
0.000805377960205
0.309760332108
0.209607362747
0.0366129875183
0.034111738205
0.0448617935181
0.0177726745605
0.00207257270813
0.0197594165802
0.0320889949799
0.0679121017456
0.0465271472931
0.0319576263428
0.054701089859
0.075202703476
0.0789768695831
0.084581136703

0.00191712379456
0.0
0.0
0.0671322345734
0.0948281288147
0.0751752853394
0.000224113464355
0.000165939331055
0.000113248825073
0.0
0.00183796882629
0.000415086746216
0.0
0.0
0.000383615493774
0.011397600174
0.191316127777
0.220800161362
0.116831302643
0.0099995136261
0.0101432800293
0.00679731369019
0.00334644317627
0.0097815990448
0.020388841629
0.0799858570099
0.26726436615
0.215803384781
0.0727214813232
0.0834574699402
0.000389099121094
0.0
0.0
0.388115167618
0.230952501297
0.210745096207
0.0576858520508
0.0151460170746
0.000196695327759
0.0186123847961
0.114333629608
0.0836341381073
0.00024151802063
0.0
1.76429748535e-05
0.000197410583496
0.00440812110901
0.00190830230713
1.12056732178e-05
0.0
0.0
0.0
0.0
0.0
0.0147411823273
0.00554132461548
0.00068998336792
1.93119049072e-05
2.76565551758e-05
0.0
0.000712871551514
0.124723672867
0.00899600982666
0.00234746932983
0.000438213348389
0.0
0.00335931777954
0.130409002304
0.0425145626068
0.0213532447815
0.00012731552124
6.96182250977e-05

0.0
0.00102019309998
0.00832962989807
0.0137152671814
0.07577085495
0.0775747299194
0.0645051002502
0.0040488243103
0.0100131034851
0.0555438995361
0.148925304413
0.127972364426
0.117260932922
0.145781517029
0.198694944382
0.144527673721
0.0613870620728
0.0827083587646
0.105303525925
0.0974678993225
0.0495264530182
0.0596566200256
0.0606112480164
0.0476250648499
0.0910170078278
0.111953258514
0.054160118103
0.000171422958374
0.0
2.38418579102e-06
1.35898590088e-05
0.0100386142731
0.101575374603
0.0741813182831
0.0798842906952
0.0651774406433
0.0521020889282
0.00860929489136
0.0
0.0
0.0
0.0209639072418
0.140529632568
0.334058523178
0.334714651108
0.00405120849609
0.0
0.0
0.0
0.000105619430542
0.128453731537
0.266372442245
0.302986383438
0.315475463867
0.310836076736
0.0349957942963
0.0187027454376
0.351701498032
0.270718336105
0.216447114944
0.00743699073792
0.0108137130737
0.00388956069946
0.00365662574768
0.0
0.00527405738831
0.111428499222
0.130096673965
0.117789268494
0.11491394043


0.00651979446411
0.00436925888062
0.000499963760376
0.000130891799927
8.225440979e-05
0.0130226612091
0.109946489334
0.0932898521423
0.081214427948
0.0463767051697
0.144839763641
0.0908191204071
0.191119670868
0.3233897686
0.215384960175
0.0351271629333
0.0320734977722
0.0120229721069
0.000690221786499
0.00128364562988
0.00112009048462
0.000976800918579
6.07967376709e-05
0.107676506042
0.121382951736
0.121288061142
0.171265125275
0.100212812424
0.0297882556915
0.035272359848
0.00916290283203
0.0349130630493
0.00637340545654
0.00427865982056
0.00476789474487
0.000734329223633
0.0162160396576
0.0178163051605
0.00120973587036
0.0
0.0
0.0
0.0
0.0
0.0
0.0393018722534
0.0189001560211
0.00823402404785
0.0016713142395
2.38418579102e-06
0.0
0.130569934845
0.168534517288
0.055624961853
0.00339245796204
0.0
0.0
0.0149574279785
0.00551295280457
6.43730163574e-06
1.6450881958e-05
0.0
0.000303745269775
0.0
0.00903177261353
0.00824689865112
0.00066876411438
0.000198364257812
0.000126838684082
0.01910

...

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.596918106079
0.489062070847
0.135085344315
0.120780706406
0.15450835228
0.140009641647
0.122779846191
0.0926749706268
0.145099163055
0.0927476882935
0.0551023483276
0.0610971450806
0.16570186615
0.820055246353
0.639838457108
0.0922501087189
0.0722117424011
0.273990631104
0.81115937233
0.924089193344
0.946314334869
0.919639825821
0.891815662384
0.934945821762
0.880988359451
0.610041618347
0.223084926605
0.142546653748
0.0897953510284
0.317754268646
0.113971233368
0.0614047050476
0.0779719352722
0.0405645370483
0.0532450675964
0.222282171249
0.527725934982
0.847865581512
0.790253162384
0.651138305664
0.148529052734
0.177059650421
0.377300262451
0.496791601181
0.428138971329
0.544779539108
0.516835451126
0.559069156647
0.75238275528
0.775332689285
0.830848693848
0.876978874207
0.926576375961
0.951842546463
0.874842166901
0.846587896347
0.648319244385
0.778723478317
0.936921596527
0.949397325516
0.910528421402
0.776846885681
0.313554048

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0555720329285
0.0554006099701
0.0564725399017
0.26521062851
0.592349052429
0.588599681854
0.654748916626
0.919017076492
0.891465425491
0.652042627335
0.837270975113
0.883917093277
0.817811727524
0.606765031815
0.93758058548
0.792821884155
0.428084135056
0.193816900253
0.0914452075958
0.0642900466919
0.0634644031525
0.065988779068
0.0909378528595
0.362102031708
0.149570226669
0.0702900886536
0.0594418048859
0.0576949119568
0.0322651863098
0.0566191673279
0.45286822319
0.804396152496
0.956453800201
0.968469619751
0.94042468071
0.940150737762
0.913673639297
0.924684762955
0.562268972397
0.30801153183
0.818186283112
0.953538417816
0.955007314682
0.942408800125
0.841466426849
0.332993745804
0.195432662964
0.733928442001
0.894309043884
0.960519552231
0.967786073685
0.942825317383
0.906740903854
0.928889036179
0.955527305603
0.963655

0.0978448390961
0.202172279358
0.116130590439
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.49768781662
0.355099916458
0.114429235458
0.102746009827
0.11114525795
0.110074281693
0.0674169063568
0.0995042324066
0.0804450511932
0.0655701160431
0.0563158988953
0.0634863376617
0.0849871635437
0.695864915848
0.892115116119
0.204345941544
0.0765292644501
0.146802186966
0.692881822586
0.974373340607
0.989005804062
0.996504068375
0.994769096375
0.984206914902
0.97859287262
0.895802259445
0.146559000015
0.0663492679596
0.0398924350739
0.0807552337646
0.231474637985
0.396786689758
0.266810894012
0.0755717754364
0.0532922744751
0.0981979370117
0.454357385635
0.755136966705
0.794511795044
0.775961637497
0.447374105453
0.596211433411
0.843273878098
0.873836755753
0.784815073013
0.7822899818

0.0132534503937
0.0169484615326
0.0251853466034
0.0522191524506
0.187872648239
0.782232761383
0.230952501297
0.0534679889679
0.0431866645813
0.0595769882202
0.123637914658
0.617978811264
0.0574605464935
0.0266101360321
0.0311515331268
0.0161952972412
0.0198998451233
0.0133421421051
0.0181975364685
0.0162410736084
0.0218858718872
0.0254094600677
0.028037071228
0.0298891067505
0.0396406650543
0.0309665203094
0.0205707550049
0.0273489952087
0.0600643157959
0.0851538181305
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.138219594955
0.100876569748
0.103455543518
0.296004533768
0.851210355759
0.75833439827
0.812453269958
0.895294904709
0.868325710297
0.466697216034
0.628551959991
0.769851922989
0.719126939774
0.154374837875
0.347004413605
0.45557808876
0.127029418945
0.0731234550476
0

KeyboardInterrupt: 